## Import libraries

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline

In [ ]:
# Create variables for your project
publish_iteration_name = "Iteration2"
project_id = "<YOUR_PROJECT_ID>"

# Create variables for your prediction resource
prediction_key = "<YOUR_KEY>"
endpoint = "<YOUR_ENDPOINT>"

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint, prediction_credentials)

In [ ]:
# Load a test image and get its dimensions
img_file = os.path.join('Images', 'Test', 'test (1).jpg')
img = Image.open(img_file)
img_height, img_width, img_ch = np.array(img).shape

In [ ]:
# Detect objects in the test image
with open(img_file, mode="rb") as test_img:
    results = predictor.detect_image(project_id, publish_iteration_name, test_img)

In [ ]:
# Create a figure to display the results
fig = plt.figure(figsize=(8, 8))
plt.axis('off')

# Display the image
draw = ImageDraw.Draw(img)

# Select line width and color for the bounding box
lineWidth = int(img_width/100)
object_colors = {
    "tomato": "yellow",
    "cucumber": "cyan",
    "pepper": "orange"
}

# Display the results
for prediction in results.predictions:
    color = 'white' # default
    if (prediction.probability*100) > 50:
        if prediction.tag_name in object_colors:
            color = object_colors[prediction.tag_name]
        left = prediction.bounding_box.left * img_width
        top = prediction.bounding_box.top * img_height
        height = prediction.bounding_box.height * img_height
        width =  prediction.bounding_box.width * img_width
        # Create a rectangle
        draw.rectangle((left, top, left+width, top+height), outline=color, width=lineWidth)
        # Display tags and probabilities
        plt.annotate(f"{prediction.tag_name}: {prediction.probability * 100 :.2f}%",(left,top), backgroundcolor=color)
plt.imshow(img)